In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download ananthu017/emotion-detection-fer/activity
! unzip emotion-detection-fer.zip

In [ ]:
! pip install wheel
! pip install pandas
! pip install -U scikit-image

In [ ]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage.transform import rescale
from skimage import exposure, io, util
from sklearn import metrics
from sklearn import svm
from sklearn.metrics import confusion_matrix

### HOG Sample

In [ ]:
image = imread('train/sad/im7.png', as_gray=True)
image_hog, image_hog_img = hog(
    image, pixels_per_cell=(4,4), 
    cells_per_block=(2, 2), 
    orientations=8, 
    visualize=True, 
    block_norm='L2-Hys')
#image_hog_img=resize(image_hog_img,(48,48))
fig, ax = plt.subplots(1, 2)
ax[0].imshow(image, cmap='gray')
ax[1].imshow(image_hog_img, cmap="gray")
print(image_hog.shape[0])
print(image.shape[0]*image.shape[1])

In [ ]:
Categories=['angry','disgusted','fearful', 'happy', 'neutral', 'sad', 'surprised']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir='train' 
#path which contains all the categories of images
for i in Categories:
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        image=resize(img_array, (48, 48))
        flat_data_arr.append(image.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x=df.iloc[:,:-1] #input data 
y=df.iloc[:,-1] #output data

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)
print('Splitted Successfully')

In [ ]:
from sklearn import svm

In [ ]:
clf = svm.SVC(kernel='rbf', gamma=0.01, C=10)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn import metrics

print("Classification report for - \n{}:\n{}\n".format(
    clf, metrics.classification_report(y_test, y_pred)))

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
 
def plot_confusion_matrix(cmx, vmax1=None, vmax2=None, vmax3=None):
    cmx_norm = 100*cmx / cmx.sum(axis=1, keepdims=True)
    cmx_zero_diag = cmx_norm.copy()
 
    np.fill_diagonal(cmx_zero_diag, 0)
 
    fig, ax = plt.subplots(ncols=3)
    fig.set_size_inches(12, 3)
    [a.set_xticks(range(len(cmx)+1)) for a in ax]
    [a.set_yticks(range(len(cmx)+1)) for a in ax]
         
    im1 = ax[0].imshow(cmx, vmax=vmax1)
    ax[0].set_title('as is')
    im2 = ax[1].imshow(cmx_norm, vmax=vmax2)
    ax[1].set_title('%')
    im3 = ax[2].imshow(cmx_zero_diag, vmax=vmax3)
    ax[2].set_title('% and 0 diagonal')
 
    dividers = [make_axes_locatable(a) for a in ax]
    cax1, cax2, cax3 = [divider.append_axes("right", size="5%", pad=0.1) 
                        for divider in dividers]
 
    fig.colorbar(im1, cax=cax1)
    fig.colorbar(im2, cax=cax2)
    fig.colorbar(im3, cax=cax3)
    fig.tight_layout()
     

In [ ]:
from sklearn.metrics import confusion_matrix

cmx_svm = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(cmx_svm)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import filters
from skimage.data import camera
from skimage.util import compare_images


image = imread('train/angry/im904.png', as_gray=True)
edge_roberts = filters.roberts(image)
edge_sobel = filters.sobel(image)
image_hog, image_hog_img = hog(
    edge_sobel, pixels_per_cell=(2,2), 
    cells_per_block=(1, 1), 
    orientations=8, 
    visualize=True, 
    block_norm='L2-Hys')

fig, axes = plt.subplots(ncols=3, sharex=True, sharey=True,
                         figsize=(8, 4))

axes[0].imshow(edge_roberts, cmap=plt.cm.gray)
axes[0].set_title('Roberts Edge Detection')

axes[1].imshow(edge_sobel, cmap=plt.cm.gray)
axes[1].set_title('Sobel Edge Detection')

axes[2].imshow(image_hog_img, cmap=plt.cm.gray)
axes[2].set_title('HOG')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.util import random_noise
from skimage import feature


# Generate noisy image of a square
image = imread('train/happy/im906.png', as_gray=True)

# Compute the Canny filter for two values of sigma
edges1 = feature.canny(image)
edges2 = feature.canny(image, sigma=1.5)

# display results
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(8, 3))

ax[0].imshow(image, cmap='gray')
ax[0].set_title('original image', fontsize=20)

ax[1].imshow(edges1, cmap='gray')
ax[1].set_title(r'Canny filter, $\sigma=1$', fontsize=20)

ax[2].imshow(edges2, cmap='gray')
ax[2].set_title(r'Canny filter, $\sigma=1.5$', fontsize=20)

for a in ax:
    a.axis('off')

fig.tight_layout()
plt.show()

# With Canny

In [ ]:
Categories=['angry','disgusted','fearful', 'happy', 'neutral', 'sad', 'surprised']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir='train' 
#path which contains all the categories of images
for i in Categories:
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    x = 0
    for img in os.listdir(path):
        x = x + 1
        if (x == 3001):
            break
        img_array=imread(os.path.join(path,img))
        image = feature.canny(img_array, sigma=1.5)
        image=resize(image,(48,48))
        flat_data_arr.append(image.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
dfCan=pd.DataFrame(flat_data) #dataframe
dfCan['Target']=target
xCan=dfCan.iloc[:,:-1] #input data 
yCan=dfCan.iloc[:,-1] #output data

In [ ]:
Categories=['angry','disgusted','fearful', 'happy', 'neutral', 'sad', 'surprised']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir='test' 
#path which contains all the categories of images
for i in Categories:
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        image = feature.canny(img_array, sigma=1.5)
        image=resize(image,(48,48))
        flat_data_arr.append(image.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
dfRTest=pd.DataFrame(flat_data) #dataframe
dfRTest['Target']=target
xRTest=dfRTest.iloc[:,:-1] #input data 
yRTest=dfRTest.iloc[:,-1] #output data

In [ ]:
from sklearn.model_selection import train_test_split
xCan_train,xCan_test,yCan_train,yCan_test=train_test_split(xCan,yCan,test_size=0.20,random_state=77,stratify=yCan)
print('Splitted Successfully')

In [ ]:
from sklearn import svm

clfCan = svm.SVC(kernel='rbf', gamma=0.01, C=10)
clfCan.fit(xCan, yCan)
yCan_pred = clfCan.predict(xRTest)

In [ ]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(yRTest, yCan_pred))

In [ ]:
print("Classification report for - \n{}:\n{}\n".format(
    clfCan, metrics.classification_report(yRTest, yCan_pred)))

In [ ]:
from sklearn.metrics import confusion_matrix

cmx_svmCan = confusion_matrix(yRTest, yCan_pred)
print(cmx_svmCan)
plot_confusion_matrix(cmx_svmCan)

In [ ]:
clfCan2 = svm.SVC(kernel='rbf', gamma=0.01, C=100)
clfCan2.fit(xCan_train, yCan_train)
yCan_pred2 = clfCan2.predict(xCan_test)
print("Accuracy:",metrics.accuracy_score(yCan_test, yCan_pred2))

In [ ]:
from sklearn.metrics import confusion_matrix

cmx_svmCan2 = confusion_matrix(yCan_test, yCan_pred2)
plot_confusion_matrix(cmx_svmCan2)

# Normal

In [ ]:
Categories=['angry','disgusted','fearful', 'happy', 'neutral', 'sad', 'surprised']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir='train' 
#path which contains all the categories of images
for i in Categories:
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    x = 0
    for img in os.listdir(path):
        x = x + 1
        if x == 1001:
          break
        img_array=imread(os.path.join(path,img))
        image=resize(img_array,(48,48))
        flat_data_arr.append(image.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x=df.iloc[:,:-1] #input data 
y=df.iloc[:,-1] #output data

In [ ]:
Categories=['angry','disgusted','fearful', 'happy', 'neutral', 'sad', 'surprised']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir='test' 
#path which contains all the categories of images
for i in Categories:
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        image=resize(img_array,(48,48))
        flat_data_arr.append(image.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
dfRTest=pd.DataFrame(flat_data) #dataframe
dfRTest['Target']=target
xRTest=dfRTest.iloc[:,:-1] #input data 
yRTest=dfRTest.iloc[:,-1] #output data

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)
print('Splitted Successfully')

In [ ]:
clf = svm.SVC(kernel='rbf', gamma=0.01, C=10)
clf.fit(x, y)
y_pred = clf.predict(xRTest)
print("Accuracy:",metrics.accuracy_score(yRTest, y_pred))

In [ ]:
y_pred = clf.predict(xRTest)
print("Accuracy:",metrics.accuracy_score(yRTest, y_pred))

In [ ]:
print(x)

In [ ]:
print("Classification report for - \n{}:\n{}\n".format(
    clf, metrics.classification_report(yRTest, y_pred)))

In [ ]:
cmx_svm = confusion_matrix(yRTest, y_pred)
print(cmx_svm)
plot_confusion_matrix(cmx_svm)

In [ ]:
clf = svm.SVC(kernel='rbf', gamma=0.01, C=10)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
print("Classification report for - \n{}:\n{}\n".format(
    clf, metrics.classification_report(y_test, y_pred)))

In [ ]:
cmx_svm = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(cmx_svm)

# Sobel

In [ ]:
CategoriesSobel=['angry','disgusted','fearful', 'happy', 'neutral', 'sad', 'surprised']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir='train' 
#path which contains all the categories of images
for i in CategoriesSobel:
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    x = 0
    for img in os.listdir(path):
        x = x + 1
        if x == 3001:
            break
        img_array=imread(os.path.join(path,img))
        image = filters.sobel(img_array)
        image=resize(image,(48,48))
        flat_data_arr.append(image.flatten())
        target_arr.append(CategoriesSobel.index(i))
    print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
dfSob=pd.DataFrame(flat_data) #dataframe
dfSob['Target']=target
xSob=dfSob.iloc[:,:-1] #input data 
ySob=dfSob.iloc[:,-1] #output data

In [ ]:
Categories=['angry','disgusted','fearful', 'happy', 'neutral', 'sad', 'surprised']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir='test' 
#path which contains all the categories of images
for i in Categories:
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        image = filters.sobel(img_array)
        image=resize(image,(48,48))
        flat_data_arr.append(image.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
dfRTest=pd.DataFrame(flat_data) #dataframe
dfRTest['Target']=target
xRTest=dfRTest.iloc[:,:-1] #input data 
yRTest=dfRTest.iloc[:,-1] #output data

In [ ]:
from sklearn.model_selection import train_test_split
xSob_train,xSob_test,ySob_train,ySob_test=train_test_split(xSob,ySob,test_size=0.20,random_state=77,stratify=ySob)
print('Splitted Successfully')

In [ ]:
clfSob = svm.SVC(kernel='rbf', gamma=0.01, C=10)
clfSob.fit(xSob, ySob)
ySob_pred = clfSob.predict(xRTest)
print("Accuracy:",metrics.accuracy_score(yRTest, ySob_pred))

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(yRTest, ySob_pred))
print("Classification report for - \n{}:\n{}\n".format(
    clfSob, metrics.classification_report(yRTest, ySob_pred)))

In [ ]:
from sklearn.metrics import confusion_matrix
cmx_svm_sobel = confusion_matrix(yRTest, ySob_pred)
print(cmx_svm_sobel)

In [ ]:
plot_confusion_matrix(cmx_svm_sobel)

In [ ]:
print("Classification report for - \n{}:\n{}\n".format(
    clfSob, metrics.classification_report(ySob_test, ySob_pred)))

In [ ]:
cmx_svm_sobel = confusion_matrix(ySob_test, ySob_pred)
plot_confusion_matrix(cmx_svm_sobel)

# HOG

In [ ]:
from skimage.feature import hog
from skimage.transform import rescale
from skimage import exposure, io, util

image = imread('train/angry/im7.png', as_gray=True)
image_hog, image_hog_img = hog(
    image, pixels_per_cell=(4,4), 
    cells_per_block=(1, 1), 
    orientations=8, 
    visualize=True, 
    block_norm='L2-Hys')
fig, ax = plt.subplots(1, 2)
ax[0].imshow(image, cmap='gray')
ax[1].imshow(image_hog_img, cmap="gray")
print(image_hog.shape[0])
print(image.shape[0]*image.shape[1])

In [ ]:
CategoriesHog=['angry','disgusted','fearful', 'happy', 'neutral', 'sad', 'surprised']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir='train' 
#path which contains all the categories of images
for i in Categories:
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    x = 0
    for img in os.listdir(path):
        x = x + 1
        if x == 2001:
          break
        img_array=imread(os.path.join(path,img))
        image_hog, image_hog_img = hog(
          img_array, pixels_per_cell=(4,4), 
          cells_per_block=(1, 1), 
          orientations=8, 
          visualize=True, 
          block_norm='L2-Hys')
        image=resize(image_hog_img,(48,48))
        flat_data_arr.append(image.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
dfHog=pd.DataFrame(flat_data) #dataframe
dfHog['Target']=target
xHog=dfHog.iloc[:,:-1] #input data 
yHog=dfHog.iloc[:,-1] #output data

In [ ]:
xHog_train,xHog_test,yHog_train,yHog_test=train_test_split(xHog,yHog,test_size=0.20,random_state=77,stratify=yHog)
print('Splitted Successfully')

In [ ]:
clfHog = svm.SVC(kernel='rbf', gamma=0.01, C=10)
clfHog.fit(xHog_train, yHog_train)
yHog_pred = clfHog.predict(xHog_test)
print("Accuracy:",metrics.accuracy_score(yHog_test, yHog_pred))

In [ ]:
print("Classification report for - \n{}:\n{}\n".format(
    clfHog, metrics.classification_report(yHog_test, yHog_pred)))

In [ ]:
cmx_svm_hog = confusion_matrix(yHog_test, yHog_pred)
plot_confusion_matrix(cmx_svm_hog)